## Install Packages

In [12]:
!pip install geopy
!pip install beautifulsoup4
!pip install lxml

     |████████████████████████████████| 122kB 7.3MB/s eta 0:00:01
     |████████████████████████████████| 5.5MB 6.2MB/s eta 0:00:01     |████████████████████████████▊   | 5.0MB 6.2MB/s eta 0:00:01


## Import Libraries

In [13]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import json
import requests
import random
import folium
import lxml
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim
from bs4 import BeautifulSoup

print('Libraries Imported!')



Libraries Imported!


## Get Data and add to DataFrame

In [60]:
html = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
table = BeautifulSoup(html,'xml').find('table')
df = pd.DataFrame( columns = ['PostalCode','Borough','Neighborhood'])
df.head()

for row in table.find_all('tr'):
        row_data = []
        for cell in row.find_all('td'):
            row_data.append(cell.text.strip())
            if len(row_data) == 3:
                df.loc[len(df)] = row_data
                
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


## Remove "Not assigned" Boroughs and change 'Not assigned' Neighborhoods to the Borough name

In [106]:
df1 = df[df['Borough'] != 'Not assigned']
df1.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [113]:
df1.loc[df1['Neighborhood'] == 'Not assigned' , 'Neighborhood'] = df1['Borough']
df1.head(10)

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


## Concatenate Neighborhoods with the same PostalCode and Borough.

In [111]:
df2 = df1.groupby(['PostalCode', 'Borough'], sort=False).agg(' , '.join)
df2.head()

,,Neighborhood
PostalCode,Borough,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Regent Park, Harbourfront"
M6A,North York,"Lawrence Manor, Lawrence Heights"
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [112]:
df2.shape

(103, 1)